<a href="https://colab.research.google.com/github/Arch-suzuki-MB/CNN-in-tensorflow/blob/main/semantic%20matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"safaebelkhyr","key":"5566c39dfa66ba4a80107aa7acf9749d"}'}

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list


ref                                                             title                                                size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
meirnizri/covid19-dataset                                       COVID-19 Dataset                                      5MB  2022-11-13 15:47:17           9674        283  1.0              
mattop/alcohol-consumption-per-capita-2016                      Alcohol Consumption Per Capita 2016                   4KB  2022-12-09 00:03:11           1008         38  1.0              
michals22/coffee-dataset                                        Coffee dataset                                       24KB  2022-12-15 20:02:12           1162         42  1.0              
thedevastator/jobs-dataset-from-glassdoor                   

In [ ]:
! kaggle competitions download -c 'us-patent-phrase-to-phrase-matching' --force

100% 682k/682k [00:00<00:00, 1.48MB/s]
100% 682k/682k [00:00<00:00, 1.48MB/s]


In [ ]:
! unzip us-patent-phrase-to-phrase-matching.zip 

Archive:  us-patent-phrase-to-phrase-matching.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 14.2 MB/s 
     |████████████████████████████████| 182 kB 79.7 MB/s 
     |████████████████████████████████| 7.6 MB 67.8 MB/s 


In [ ]:
import numpy as np
import tensorflow as tf 
import pandas as pd
import transformers 

Configuration

In [ ]:
max_length=128
batch_size=32
epochs=2

#labels in dataset 
labels=['1.0' ,'0.75' ,'0.5' ,'0.25','0.0' ]

# Load the data 

In [ ]:
from sklearn.model_selection import train_test_split

data_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")

In [ ]:
#dataset overview 
data_df.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [ ]:
test_df.head()

,id,anchor,target,context
0,4112d61851461f60,opc drum,inorganic photoconductor drum,G02
1,09e418c93a776564,adjust gas flow,altering gas flow,F23
2,36baf228038e314b,lower trunnion,lower locating,B60
3,1f37ead645e7f0c8,cap component,upper portion,D06
4,71a5b6ad068d531f,neural stimulation,artificial neural network,H04


In [ ]:
data_df=data_df.drop(['id','context'],axis=1)
test_df=test_df.drop(['id','context'],axis=1)
data_df.head()

,anchor,target,score
0,abatement,abatement of pollution,0.50
1,abatement,act of abating,0.75
2,abatement,active catalyst,0.25
3,abatement,eliminating process,0.50
4,abatement,forest region,0.00


In [ ]:
train_df, valid_df = train_test_split(data_df, test_size=0.2)

#shape of the data 
print('the whole data shape is : ',data_df.shape)
print('the train data shape is : ',train_df.shape)
print('the val data shape is : ',valid_df.shape)

the whole data shape is :  (36473, 3)
the train data shape is :  (29178, 3)
the val data shape is :  (7295, 3)


## Preprocessing

In [ ]:
print("Number od missing values")
print(train_df.isnull().sum())
train_df.dropna(axis=0,inplace=True)

Number od missing values
anchor    0
target    0
score     0
dtype: int64


In [ ]:
#Distribution of our training score
print('train distribution ')
print(train_df.score.value_counts() )
print('\n validation  distribution ')
print(valid_df.score.value_counts() )

train distribution 
0.50    9819
0.25    9248
0.00    5941
0.75    3229
1.00     941
Name: score, dtype: int64

 validation  distribution 
0.50    2481
0.25    2271
0.00    1530
0.75     800
1.00     213
Name: score, dtype: int64


In [ ]:
train_df =(train_df.sample(frac=1.0,random_state=42).reset_index(drop=True))
valid_df =(valid_df.sample(frac=1.0,random_state=42).reset_index(drop=True))

In [ ]:
train_df['score']=train_df["score"].apply(
    lambda x:0 if x=="0.0" else 1 if x=="0.25" else 2 if x=="0.5" else 3 if x=="0.75" else 4 
    )
y_train=tf.keras.utils.to_categorical(train_df.score,num_classes=5)



valid_df['score']=valid_df["score"].apply(
    lambda x:0 if x=='0.0' else 1 if x=='0.25' else 2 if x=='0.5' else 3 if x=='0.75' else 4 
)
y_val=tf.keras.utils.to_categorical(valid_df.score,num_classes=5)

#labels=['1.0' ,'0.75' ,'0.5' ,'0.25','0.0' ]

## Keras Custom DataGenerator

In [ ]:

class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of premise and hypothesis input sentences.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to incude the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(self,sentence_pairs,labels,batch_size=batch_size,shuffle=True,include_targets=True,):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=max_length,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)



## Build the model

In [ ]:
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="input_ids"
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("bert-base-uncased")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

    bert_output = bert_model.bert(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True)
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)
    output = tf.keras.layers.Dense(3, activation="softmax")(dropout)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )

    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss="categorical_crossentropy",
        metrics=["acc"],
    )


print(f"Strategy: {strategy}")
model.summary()

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]